**Name**: Ali KÖMÜRCÜ

**Student Id**: 2380699

In [286]:
import numpy as np
import cv2 as cv
from scipy import ndimage
IMAGES_PATH = "images/"
OUTPUT_PATH = "outputs/"

In [287]:
# Read images
agac = cv.imread(IMAGES_PATH + "agac.png", cv.IMREAD_GRAYSCALE)
agacRgb = cv.imread(IMAGES_PATH + "agac.png")
agacrotated = cv.imread(IMAGES_PATH + "agacrotated.png", cv.IMREAD_GRAYSCALE)
agacrotatedRgb = cv.imread(IMAGES_PATH + "agacrotated.png")

chessboard = cv.imread(IMAGES_PATH + "chessboard.png", cv.IMREAD_GRAYSCALE)
chessboardRgb = cv.imread(IMAGES_PATH + "chessboard.png")
chessboardrotated = cv.imread(IMAGES_PATH + "chessboardrotated.png", cv.IMREAD_GRAYSCALE)
chessboardrotatedRgb = cv.imread(IMAGES_PATH + "chessboardrotated.png")

lab = cv.imread(IMAGES_PATH + "lab.png", cv.IMREAD_GRAYSCALE)
labRgb = cv.imread(IMAGES_PATH + "lab.png")
labrotated = cv.imread(IMAGES_PATH + "labrotated.png", cv.IMREAD_GRAYSCALE)
labrotatedRgb = cv.imread(IMAGES_PATH + "labrotated.png")


In [288]:
def max_n_dots(image, imageRgb, max_dots=10, window_size=10):
    index_10 = np.argpartition(image.ravel(), -max_dots)[-max_dots:]
    x_index = index_10 // image.shape[0]
    y_index = index_10 % image.shape[1]
    for i in range(len(x_index)):
        x = x_index[i]
        y = y_index[i]
        cv.circle(imageRgb, (y, x), 1, (0, 0, 255), 2)
    return imageRgb

In [289]:
def nonmax_suppression(output, nonmax=5):
    hf = nonmax // 2
    for x in range(hf, output.shape[0]-hf): # x direction, -u is here for not going beyond the borders
        for y in range(hf, output.shape[1]-hf):   # y direction. same for -v
            neighborhood = output[x-hf:x+hf+1, y-hf:y+hf+1]
            maxima = np.argmax(neighborhood)
            xind = maxima // nonmax
            yind = maxima % nonmax
            # neighborhood = neighborhood.ravel()
            for i in range(nonmax):
                for j in range(nonmax):
                    if i != xind or j != yind:
                        neighborhood[i,j] = 0
    return output

### Task 1: Naive Formula + Uniform Weighting

In [290]:
def task1(image, imageRgb, wsize):    # window = wsize x wsize
    img = image.copy()
    imgRgb = imageRgb.copy()
    output = np.zeros((img.shape[0], img.shape[1]), np.float32)
    hf = wsize//2
    for x in range(hf+1, img.shape[0]-hf-1): # x direction, -u is here for not going beyond the borders
        for y in range(hf+1, img.shape[1]-hf-1):   # y direction. same for -v
            x_ind1, x_ind2 = x-hf, x+hf+1
            y_ind1, y_ind2 = y-hf, y+hf+1
            window = img[x_ind1:x_ind2, y_ind1:y_ind2]  # exclude hf + 1
            E = 0
            for u in [-1,0,1]:
                for v in [-1,0,1]:
                    if u == 0 and v == 0:
                        continue
                    E += np.sum(np.square(img[x_ind1+u:x_ind2+u, y_ind1+v:y_ind2+v] - window))
            output[x, y] = E
    return max_n_dots(output, imgRgb)

In [291]:
aret = task1(agac, agacRgb, wsize=3)
cv.imwrite(OUTPUT_PATH + "task1_agac.png", aret)

arret = task1(agacrotated, agacrotatedRgb, wsize=3)
cv.imwrite(OUTPUT_PATH + "task1_agacrotated.png", arret)

cret = task1(chessboard, chessboardRgb, wsize=3)
cv.imwrite(OUTPUT_PATH + "task1_chessboard.png", cret)

crret = task1(chessboardrotated, chessboardrotatedRgb, wsize=3)
cv.imwrite(OUTPUT_PATH + "task1_chessboardrotated.png", crret)

lret = task1(lab, labRgb, wsize=3)
cv.imwrite(OUTPUT_PATH + "task1_lab.png", lret)

lrret = task1(labrotated, labrotatedRgb, wsize=3)
cv.imwrite(OUTPUT_PATH + "task1_labrotated.png", lrret)

True

### TASK1 Discussion

- Formula in slide 24 is implemented.

- **Time elpased:** 26.8 s

![agac](outputs/task1_agac.png) ![agacRotated](outputs/task1_agacrotated.png)


![chessboard](outputs/task1_chessboard.png) ![chessboardrotated](outputs/task1_chessboardrotated.png)


![lab](outputs/task1_lab.png) ![labrotated](outputs/task1_labrotated.png)



### Task2: Naive Formula + Uniform Weighting + Non-Maximum Suppression

In [292]:
def task2(image, imageRgb, wsize, nonmax=5):    # window = wsize x wsize
    # construct new images from the input image both rgb and grayscale
    img = np.array(image)
    imgRgb = np.array(imageRgb)
    # create an output image
    output = np.zeros((img.shape[0], img.shape[1]), np.float32)
    hf = wsize//2   # half of the window size
    for x in range(hf+1, img.shape[0]-hf-1):        # x direction, hf logic is here for not going beyond the borders
        for y in range(hf+1, img.shape[1]-hf-1):    # y direction. same for hf
            x_ind1, x_ind2 = x-hf, x+hf+1
            y_ind1, y_ind2 = y-hf, y+hf+1
            window = img[x_ind1:x_ind2, y_ind1:y_ind2]  # exclude hf + 1
            E = 0
			# Slide in each 8 directions and calculate the sum of the squared differences
            for u in [-1,0,1]:
                for v in [-1,0,1]:
                    if u == 0 and v == 0:
                        continue
                    E += np.sum(np.square(img[x_ind1+u:x_ind2+u, y_ind1+v:y_ind2+v] - window))
            output[x, y] = E
    # # # Non-maximum suppression
    output = nonmax_suppression(output)
    return max_n_dots(output, imgRgb)

In [293]:
aret = task2(agac, agacRgb, wsize=3)
cv.imwrite(OUTPUT_PATH + "task2_agac.png", aret)

arret = task2(agacrotated, agacrotatedRgb, wsize=3)
cv.imwrite(OUTPUT_PATH + "task2_agacrotated.png", arret)

cret = task2(chessboard, chessboardRgb, wsize=3)
cv.imwrite(OUTPUT_PATH + "task2_chessboard.png", cret)

crret = task2(chessboardrotated, chessboardrotatedRgb, wsize=3)
cv.imwrite(OUTPUT_PATH + "task2_chessboardrotated.png", crret)

lret = task2(lab, labRgb, wsize=3)
cv.imwrite(OUTPUT_PATH + "task2_lab.png", lret)

lrret = task2(labrotated, labrotatedRgb, wsize=3)
cv.imwrite(OUTPUT_PATH + "task2_labrotated.png", lrret)

True

### TASK2 Discussion

- Formula in slide 24 is implemented.

- Nonmaximum suppression enhanced the algorithm in terms of finding more edges.

- **Time elpased:** 31.2 s

![agac](outputs/task2_agac.png) ![agacRotated](outputs/task2_agacrotated.png)


![chessboard](outputs/task2_chessboard.png) ![chessboardrotated](outputs/task2_chessboardrotated.png)


![lab](outputs/task2_lab.png) ![labrotated](outputs/task2_labrotated.png)



### Task3: Taylor's Approximation + Uniform Weighting + Non-Maximum Suppression

In [294]:
def task3(image, imageRgb, wsize=5, nonmax=5, k=0.04):
    # construct new images from the input image both rgb and grayscale
	img = np.array(image)
	imgRgb = np.array(imageRgb)
	# find the gradients with sobel filter
	Ix = cv.Sobel(img, cv.CV_64F, 1, 0)
	Iy = cv.Sobel(img, cv.CV_64F, 0, 1)
	Ixx = Ix * Ix
	Iyy = Iy * Iy
	Ixy = Ix * Iy
	# create an output image
	output = np.zeros((img.shape[0], img.shape[1]), np.float32)
	hf = wsize//2	# half of the window size
	for x in range(hf+1, img.shape[0]-hf-1):
		for y in range(hf+1, img.shape[1]-hf-1):
			x_ind1, x_ind2 = x-hf, x+hf+1 	# indices for the x direction
			y_ind1, y_ind2 = y-hf, y+hf+1	# indices for the y direction
			IxIx = Ixx[x_ind1:x_ind2, y_ind1:y_ind2]
			IyIy = Iyy[x_ind1:x_ind2, y_ind1:y_ind2]
			IxIy = Ixy[x_ind1:x_ind2, y_ind1:y_ind2]
			H = np.array([[np.sum(IxIx), np.sum(IxIy)], [np.sum(IxIy), np.sum(IyIy)]])  # compute the sum of the squares of the gradients
			E = 0
			# Slide in each 8 directions and calculate the formula
			for u in [-1,0,1]:
				for v in [-1,0,1]:
					if u == 0 and v == 0:
						continue
					# E += sum ( [u,v] H [u,v]^T )
					E += np.matmul(
						[u,v], (np.matmul(H, 
											np.transpose([u,v]))))
			output[x, y] = E
	# # Non-maximum suppression
	output = nonmax_suppression(output)
	return max_n_dots(output, imgRgb)

In [295]:
aret = task3(agac, agacRgb, nonmax=5)
cv.imwrite(OUTPUT_PATH + "task3_agac.png", aret)

arret = task3(agacrotated, agacrotatedRgb)
cv.imwrite(OUTPUT_PATH + "task3_agacrotated.png", arret)

cret = task3(chessboard, chessboardRgb)
cv.imwrite(OUTPUT_PATH + "task3_chessboard.png", cret)

crret = task3(chessboardrotated, chessboardrotatedRgb)
cv.imwrite(OUTPUT_PATH + "task3_chessboardrotated.png", crret)

lret = task3(lab, labRgb)
cv.imwrite(OUTPUT_PATH + "task3_lab.png", lret)

lrret = task3(labrotated, labrotatedRgb)
cv.imwrite(OUTPUT_PATH + "task3_labrotated.png", lrret)


True

### TASK3 Discussion

- Formula in slide 29 is implemented.

- Taylor's approximation of the formula is used.

- Nonmaximum suppression used.

- **Time elpased:** 44.8 s

![agac](outputs/task3_agac.png) ![agacRotated](outputs/task3_agacrotated.png)


![chessboard](outputs/task3_chessboard.png) ![chessboardrotated](outputs/task3_chessboardrotated.png)


![lab](outputs/task3_lab.png) ![labrotated](outputs/task3_labrotated.png)



### Task 4: Smaller Eigenvalue as corner score + Uniform Weighting + Non-Maximum Suppression

In [296]:
def task4(image, imageRgb, wsize=5, k=0.04, threshold=0.1):
    # construct new images from the input image both rgb and grayscale
	img = np.array(image)
	imgRgb = np.array(imageRgb)
	# find the gradients with sobel filter
	Ix = cv.Sobel(img, cv.CV_64F, 1, 0)
	Iy = cv.Sobel(img, cv.CV_64F, 0, 1)
	Ixx = Ix * Ix
	Iyy = Iy * Iy
	Ixy = Ix * Iy
	# create an output image
	output = np.zeros((img.shape[0], img.shape[1]), np.float32)
	hf = wsize//2	# half of the window size
	for x in range(hf+1, img.shape[0]-hf-1):
		for y in range(hf+1, img.shape[1]-hf-1):
			x_ind1, x_ind2 = x-hf, x+hf+1 	# indices for the x direction
			y_ind1, y_ind2 = y-hf, y+hf+1	# indices for the y direction
			IxIx = Ixx[x_ind1:x_ind2, y_ind1:y_ind2]
			IyIy = Iyy[x_ind1:x_ind2, y_ind1:y_ind2]
			IxIy = Ixy[x_ind1:x_ind2, y_ind1:y_ind2]
			H = np.array([[np.sum(IxIx), np.sum(IxIy)], [np.sum(IxIy), np.sum(IyIy)]])  # compute the sum of the squares of the gradients
			# No need to look for 8 regions, since we are only interested in the smaller eigenvalue
			eig = np.min(np.linalg.eigvals(H)) # smallest eigenvalue of H
			output[x, y] = eig if eig > threshold else 0	
	# Non-maximum suppression
	output = nonmax_suppression(output)
	return max_n_dots(output, imgRgb)	# return R

In [297]:
aret = task4(agac, agacRgb)
cv.imwrite(OUTPUT_PATH + "task4_agac.png", aret)

arret = task4(agacrotated, agacrotatedRgb)
cv.imwrite(OUTPUT_PATH + "task4_agacrotated.png", arret)

cret = task4(chessboard, chessboardRgb)
cv.imwrite(OUTPUT_PATH + "task4_chessboard.png", cret)

crret = task4(chessboardrotated, chessboardrotatedRgb)
cv.imwrite(OUTPUT_PATH + "task4_chessboardrotated.png", crret)

lret = task4(lab, labRgb)
cv.imwrite(OUTPUT_PATH + "task4_lab.png", lret)

lrret = task4(labrotated, labrotatedRgb)
cv.imwrite(OUTPUT_PATH + "task4_labrotated.png", lrret)

True

### TASK4 Discussion

- Formula in slide 40 is implemented.

- H matrix of each pixel is found.

- Minimum eigenvalue of each matrix is used to thresholding.

- Nonmaximum suppression used.

- **Time elpased:** 21.2 s

![agac](outputs/task4_agac.png) ![agacRotated](outputs/task4_agacrotated.png)


![chessboard](outputs/task4_chessboard.png) ![chessboardrotated](outputs/task4_chessboardrotated.png)


![lab](outputs/task4_lab.png) ![labrotated](outputs/task4_labrotated.png)



### Task 5: R function as corner score + Uniform Weighting + Non-Maximum Suppression

In [298]:
def task5(image, imageRgb, wsize=5, k=0.04, threshold=0.01):
    # construct new images from the input image both rgb and grayscale
	img = np.array(image)
	imgRgb = np.array(imageRgb)
	# find gradients with sobel filter
	Ix = cv.Sobel(img, cv.CV_64F, 1, 0)
	Iy = cv.Sobel(img, cv.CV_64F, 0, 1)
	Ixx = Ix * Ix
	Iyy = Iy * Iy
	Ixy = Ix * Iy
	# create an output image
	output = np.zeros((img.shape[0], img.shape[1]), np.float32)
	hf = wsize//2	# half of the window size
	for x in range(hf+1, img.shape[0]-hf-1):
		for y in range(hf+1, img.shape[1]-hf-1):
			x_ind1, x_ind2 = x-hf, x+hf+1	# indices for the x direction
			y_ind1, y_ind2 = y-hf, y+hf+1	# indices for the y direction
			IxIx = np.sum(Ixx[x_ind1:x_ind2, y_ind1:y_ind2])
			IyIy = np.sum(Iyy[x_ind1:x_ind2, y_ind1:y_ind2])
			IxIy = np.sum(Ixy[x_ind1:x_ind2, y_ind1:y_ind2])
			# det(h) - k * trace(H)^2
			R = (IxIx * IyIy - IxIy * IxIy) - k * (IxIx + IyIy)**2
			output[x, y] = R if R > threshold else 0
	# Non-maximum suppression
	output = nonmax_suppression(output)
	return max_n_dots(output, imgRgb)

In [299]:
aret = task5(agac, agacRgb)
cv.imwrite(OUTPUT_PATH + "task5_agac.png", aret)

arret = task5(agacrotated, agacrotatedRgb)
cv.imwrite(OUTPUT_PATH + "task5_agacrotated.png", arret)

cret = task5(chessboard, chessboardRgb)
cv.imwrite(OUTPUT_PATH + "task5_chessboard.png", cret)

crret = task5(chessboardrotated, chessboardrotatedRgb)
cv.imwrite(OUTPUT_PATH + "task5_chessboardrotated.png", crret)

lret = task5(lab, labRgb)
cv.imwrite(OUTPUT_PATH + "task5_lab.png", lret)

lrret = task5(labrotated, labrotatedRgb)
cv.imwrite(OUTPUT_PATH + "task5_labrotated.png", lrret)

True

### TASK5 Discussion

- Formula in slide 41 is implemented.

- H matrix of each pixel is found.

- R function is used to thresholding.

- Nonmaximum suppression used.

- **Time elpased:** 10.8 s

![agac](outputs/task5_agac.png) ![agacRotated](outputs/task5_agacrotated.png)


![chessboard](outputs/task5_chessboard.png) ![chessboardrotated](outputs/task5_chessboardrotated.png)


![lab](outputs/task5_lab.png) ![labrotated](outputs/task5_labrotated.png)



### Task 6: R function with fast windowing based on fitering + Uniform Weighting + Non-Maximum Suppression

In [300]:
def task6(image, imageRgb, wsize=5, k=0.04):
    # construct new images from the input image both rgb and grayscale
	img = np.array(image)
	imgRgb = np.array(imageRgb)
	# find gradients with sobel filter
	Ix = cv.Sobel(img, cv.CV_64F, 1, 0)
	Iy = cv.Sobel(img, cv.CV_64F, 0, 1)
	# create an output image
	output = np.zeros((img.shape[0], img.shape[1]), np.float32)
	# create a 2x2 matrix of the form:
	# Ix^2   IxIy
	# IxIy   Iy^2
	Ixx = Ix * Ix
	Iyy = Iy * Iy
	Ixy = Ix * Iy
	# apply averaging filter to the gradients
	Ixx = cv.filter2D(Ixx, -1, np.ones((wsize, wsize)))
	Iyy = cv.filter2D(Iyy, -1, np.ones((wsize, wsize)))
	Ixy = cv.filter2D(Ixy, -1, np.ones((wsize, wsize)))
	# R = det(h) - k * trace(H)^2
	output = Ixx * Iyy - Ixy * Ixy - k * (Ixx + Iyy) * (Ixx + Iyy)
	# Non-maximum suppression
	output = nonmax_suppression(output)
	return max_n_dots(output, imgRgb)	# return R

In [301]:
aret = task6(agac, agacRgb)
cv.imwrite(OUTPUT_PATH + "task6_agac.png", aret)

arret = task6(agacrotated, agacrotatedRgb)
cv.imwrite(OUTPUT_PATH + "task6_agacrotated.png", arret)

cret = task6(chessboard, chessboardRgb)
cv.imwrite(OUTPUT_PATH + "task6_chessboard.png", cret)

crret = task6(chessboardrotated, chessboardrotatedRgb)
cv.imwrite(OUTPUT_PATH + "task6_chessboardrotated.png", crret)

lret = task6(lab, labRgb)
cv.imwrite(OUTPUT_PATH + "task6_lab.png", lret)

lrret = task6(labrotated, labrotatedRgb)
cv.imwrite(OUTPUT_PATH + "task6_labrotated.png", lrret)

True

### TASK6 Discussion

- Formula in slide 51-52 is implemented with option 1.

- One H matrix of whole image is found.

- Uniform averaging implemented on H matrix.

- R function is used to thresholding.

- Nonmaximum suppression used.

- **Time elpased:** 4.6 s

![agac](outputs/task6_agac.png) ![agacRotated](outputs/task6_agacrotated.png)


![chessboard](outputs/task6_chessboard.png) ![chessboardrotated](outputs/task6_chessboardrotated.png)


![lab](outputs/task6_lab.png) ![labrotated](outputs/task6_labrotated.png)



### Task 7: R function with fast windowing based on fitering + Gaussian Weighting + Non-Maximum Suppression

In [302]:
def task7(image, imageRgb, wsize=5, k=0.04):
    # construct new images from the input image both rgb and grayscale
    img = np.array(image)
    imgRgb = np.array(imageRgb)
    # find gradients with sobel filter
    Ix = cv.Sobel(img, cv.CV_64F, 1, 0)
    Iy = cv.Sobel(img, cv.CV_64F, 0, 1)
    # create an output image
    output = np.zeros((img.shape[0], img.shape[1]), np.float32)
    # create a 2x2 matrix of the form:
    # Ix^2   IxIy
    # IxIy   Iy^2
    Ixx = Ix * Ix
    Iyy = Iy * Iy
    Ixy = Ix * Iy
    # apply gaussian filter to the gradients
    Ixx = cv.GaussianBlur(Ixx, (wsize, wsize), 0)
    Iyy = cv.GaussianBlur(Iyy, (wsize, wsize), 0)
    Ixy = cv.GaussianBlur(Ixy, (wsize, wsize), 0)
    # R = det(h) - k * trace(H)^2
    output = Ixx * Iyy - Ixy * Ixy - k * (Ixx + Iyy) * (Ixx + Iyy)
    # Non-maximum suppression
    output = nonmax_suppression(output)
    return max_n_dots(output, imgRgb)	# return R

In [303]:
aret = task7(agac, agacRgb)
cv.imwrite(OUTPUT_PATH + "task7_agac.png", aret)

arret = task7(agacrotated, agacrotatedRgb)
cv.imwrite(OUTPUT_PATH + "task7_agacrotated.png", arret)

cret = task7(chessboard, chessboardRgb)
cv.imwrite(OUTPUT_PATH + "task7_chessboard.png", cret)

crret = task7(chessboardrotated, chessboardrotatedRgb)
cv.imwrite(OUTPUT_PATH + "task7_chessboardrotated.png", crret)

lret = task7(lab, labRgb)
cv.imwrite(OUTPUT_PATH + "task7_lab.png", lret)

lrret = task7(labrotated, labrotatedRgb)
cv.imwrite(OUTPUT_PATH + "task7_labrotated.png", lrret)

True

### TASK7 Discussion

- Formula in slide 51-52 is implemented with option 2.

- One H matrix of whole image is found.

- Gaussian averaging implemented on H matrix.

- R function is used to thresholding.

- Nonmaximum suppression used.

- **Time elpased:** 4.6 s

![agac](outputs/task7_agac.png) ![agacRotated](outputs/task7_agacrotated.png)


![chessboard](outputs/task7_chessboard.png) ![chessboardrotated](outputs/task7_chessboardrotated.png)


![lab](outputs/task7_lab.png) ![labrotated](outputs/task7_labrotated.png)



### Conclusion

We obtained that by increasing in the tasks our algorithms became **more efficient** and **faster**.

Although nonmaximum suppression brings **more computation cost**, we can say that nonmaximum suppression gives us **more precise** corners rather than not using nonmaximum suppression.

We can say that ***Taylor's approximation*** increases the number of operations to make. Instead of taking square differences we make matrix multiplication. Hence the process **decelerates**. However, in my opinion, our algorithm has a better structure now, with matrix operations. In the following steps, by using Taylor's approach we will derive more efficient algorithms.

We can say that using ***minimum eigenvalue*** *(i.e. Shi-Tomasi Method)* to detect corners is **more efficient** than using only Taylor's approximation in terms of computation and preciseness.

We can say that using ***R function*** as corner score is **more precise** and **faster** method than ***Shi-Tomasi*** method.

Finally, we can say that using ***fast windowing*** method **accelerates** te process at most.